In [143]:
import pandas as pd
from pathlib import Path
from tqdm import tqdm

In [144]:
# from process.water import water_process1

In [145]:
data_dir = Path.cwd().parents[2] / 'original_data/train'
rain = pd.read_csv(data_dir / 'rainfall' / 'data.csv')
rain_st = pd.read_csv(data_dir / 'rainfall' / 'stations.csv')
tide = pd.read_csv(data_dir / 'tidelevel' / 'data.csv')
tide_st = pd.read_csv(data_dir / 'tidelevel' / 'stations.csv')
water = pd.read_csv(data_dir / 'waterlevel' / 'data.csv')
water_st = pd.read_csv(data_dir / 'waterlevel' / 'stations.csv')
dam = pd.read_csv(data_dir / 'dam.csv')

In [146]:
import pandas as pd
import numpy as np
import gc
import math

def rain_process1(rain, rain_st):
    ###################
    # data.csv側の処理 #
    ###################

    ##### 全体でdrop_duplicates
    rain.drop_duplicates(inplace=True)
    
    ##### (date, statition, city)が重複している行 -> 
    #####     その行の中でfloatが最も含まれている行を採用
    # (date, station, city)でグループを作った時、2行以上ある場合は重複なのでそのindexだけ取り出す
    nunique_date_st_city = rain.groupby(['date', 'station', 'city']).nunique()
    nunique_date_st_city['max_count'] = nunique_date_st_city.max(axis=1)

    dup_date_st_city = nunique_date_st_city.query('max_count >= 2')[['max_count']]
    dup_date_st_city_idx =  dup_date_st_city.index
    # 重複した行それぞれに含まれる値でfloatである値をカウント
    dup_date_st_city_df = rain.set_index(['date', 'station', 'city']).loc[dup_date_st_city_idx]
    dup_date_st_city_df['num_count'] = dup_date_st_city_df.apply(lambda x: \
        24 - pd.to_numeric(x, errors='coerce').isnull().sum(),axis=1)
    # floatが最も多い1行だけをとり出してconcat
    concat_df = None
    for _, df in dup_date_st_city_df.groupby(['date', 'station', 'city']):
        df = df.sort_values('num_count', na_position='first', ascending=False).iloc[0:1, :]
        if concat_df is None:
            concat_df = df.copy()
        else:
            concat_df = pd.concat([concat_df, df], axis=0)
    concat_df.drop('num_count', inplace=True, axis=1)

    # 重複していない行たちとconcat
    unique_date_st_city = nunique_date_st_city.query('max_count == 1')
    unique_date_st_city =  unique_date_st_city.index
    unique_date_st_city_df = rain.set_index(['date', 'station', 'city']).loc[unique_date_st_city]
    rain = pd.concat([unique_date_st_city_df, concat_df])

    del nunique_date_st_city, dup_date_st_city, dup_date_st_city_df, \
        unique_date_st_city, unique_date_st_city_df, concat_df, df
    gc.collect()

    rain.reset_index(inplace=True)
    rain.sort_values(['date', 'station', 'city'], inplace=True)

    ##### (おそらく)同じstationである行の値をマージ
    # 観測日数が31日のstationは、そのstation名に(電)のついたものと同じstationと考えられるのでマージ
    for st in rain['station'].value_counts()[rain['station'].value_counts() == 31].index:
        st_ = st + '(電)'
        # 変更前後の２つのstationに含まれるデータに日付の重なりがなければマージ
        if len(rain.query('station in (@st, @st_)')) == rain.query('station in (@st, @st_)')['date'].nunique():
            rain.loc[rain['station'] == st, 'station'] = st_

    # station.csvには存在しないstationで、(国)をつけたものなら存在するものはおなじstationとしてマージ
    for st in set(rain['station'].unique()) - set(rain_st['観測所名称'].unique()):
        bool_ = rain_st['観測所名称'].str.contains(st)
        if (bool_).any():
            st_ = rain_st[bool_]['観測所名称'].iloc[-1]
            if f'{st}(国)' == st_:
                # 変更前後の２つのstationに含まれるデータに日付の重なりがなければマージ
                if len(rain.query('station in (@st, @st_)')) == rain.query('station in (@st, @st_)')['date'].nunique():
                    rain.loc[rain['station'] == st, 'station'] = st_
    
    ######################
    # station.csv側の処理 #
    ######################

    ##### station名に(砂防)が含まれているものは入力時使用も0であり、ないものとマージできる
    rain_st.loc[:, '観測所名称'] = rain_st['観測所名称'].str.replace(r'\(砂防\)', '')

    ###################
    # データベースを作成 #
    ###################
    
    # idに(station, city)を対応させたテーブルを作る
    keys = rain.groupby(['station', 'city']).count().index
    rain_db = pd.DataFrame(index=keys).reset_index()
    rain_db['id'] = range(len(rain_db))
    rain_db = rain_db.reindex(columns=['id', 'station', 'city'])

    # column名に変更を加える
    rain_st = rain_st.rename(columns={'観測所名称': 'station', '市町': 'city'})

    # station.csvのcityがnanのもののうち、data.csvから埋められるものは埋める
    for data in rain_st.iterrows(): # stationを一列ずつ取り出す
        city = data[1]['city'] # cityを取り出す
        if isinstance(city, float) and math.isnan(city): # そのcityがnanの時のみ
            st = data[1]['station']
            city = rain.query('station==@st')['city'].unique()[0] # data.csvからそのstationを検索してなんのcityかをみる
            rain_st.loc[(rain_st['station'] == st), 'city'] = city
    
    # data.csvの(station, city)をidに置き換える
    rain = rain_db.merge(rain, on=['station', 'city'], how='left')
    rain.drop(['station', 'city'], axis=1, inplace=True)

    rain_st = rain_db.merge(rain_st, on=['station', 'city'], how='left')
    rain_st['入力時使用'] = rain_st['入力時使用'].fillna(0.0)

    return rain, rain_st

In [164]:
rain_st_p1

,id,station,city,フリガナ,水系名,河川名,データ所管,住所,緯度,経度,事務所,入力時使用
0,0,くすの木台,広島市安佐北区,クスノキダイ,太田川,吉山川,国土交通省,広島市安佐北区安佐町大字くすの木台4-2,34.505278,132.383611,NaN,0.0
1,1,くすの木台(国),広島市安佐北区,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,2,もみのき,廿日市市,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,3,ゆめランド,三次市,ユメランド,江の川,布野川,砂防課,三次市布野町下布野字大谷６６１－１,34.855028,132.797500,NaN,1.0
4,4,七曲(国),北広島町,ナナマガリ,太田川,西宗川,国土交通省,山県郡北広島町吉木字七曲4779-1,34.615000,132.384722,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
467,467,魚切ダム,広島市佐伯区,ウオキリダム,八幡川,八幡川,河川課\nダムG,広島市佐伯区五日市町大字上河内上山之内トチガ谷９９８－７,34.428333,132.317778,NaN,1.0
468,468,黒峠,安芸太田町,クロダオ,太田川,西宗川,砂防課,山県郡安芸太田町大字穴字黒峠５９１番地の１,34.569278,132.366111,NaN,1.0
469,469,黒川,世羅町,クロガワ,江の川,美波羅川,砂防課,世羅郡世羅町大字黒川５２７－１７,34.643500,132.898056,NaN,1.0
470,470,黒瀬町,東広島市,クロセチョウ,黒瀬川,神洗川,河川課,東広島市黒瀬町字丸山１３３３ 東広島市役所黒瀬支所内,34.324444,132.667500,NaN,1.0


In [147]:
rain_p1, rain_st_p1 = rain_process1(rain, rain_st)

/var/folders/s8/kn1s0g_s407d23sqsmvzcql80000gn/T/ipykernel_39382/2105682670.py:59: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  bool_ = rain_st['観測所名称'].str.contains(st)
/var/folders/s8/kn1s0g_s407d23sqsmvzcql80000gn/T/ipykernel_39382/2105682670.py:72: FutureWarning: The default value of regex will change from True to False in a future version.
  rain_st.loc[:, '観測所名称'] = rain_st['観測所名称'].str.replace(r'\(砂防\)', '')


In [148]:
rain_db01 = pd.read_csv(Path.cwd().parents[2] / 'data/database01/rain_data.csv')
rain_st_db01 = pd.read_csv(Path.cwd().parents[2] / 'data/database01/rain_station.csv')
print(rain_p1.equals(rain_db01))
print(rain_st_p1.equals(rain_st_db01))

True
True


In [185]:
def water_process1(water, water_st):

    water['station'] = water['station'].str.replace(r'\(電\)', '')

    for st in set(water['station'].unique()) - set(water_st['観測所名称'].unique()):
        bool_ = water_st['観測所名称'].str.contains(st)
        if (bool_).any():
            st_ = water_st[bool_]['観測所名称'].iloc[-1]
            if f'{st}(国)' == st_:
                if len(water.query('station in (@st, @st_)')) == water.query('station in (@st, @st_)')['date'].nunique():
                    water.loc[water['station'] == st, 'station'] = st_
    
    water.loc[water['station'] == '山手', 'station'] = '山手(国)'

    # print(set(water_st['観測所名称'].unique()) - set(water['station'].unique()))
    water.loc[water['river']=='太田川放水路', 'river'] = '太田川\\n放水路'
    
    keys = water.groupby(['station', 'river']).count().index
    water_db = pd.DataFrame(index=keys).reset_index()
    water_db['id'] = range(len(water_db))
    water_db = water_db.reindex(columns=['id', 'station', 'river'])

    water = water_db.merge(water, on=['station', 'river'], how='left')
    water.drop(['station', 'river'], axis=1, inplace=True)

    water_st = water_st.rename(columns={'観測所名称': 'station', '河川名': 'river'})
    water_st = water_db.merge(water_st, on=['station', 'river'], how='left')

    return water, water_st

In [186]:
water_p1, water_st_p1 = water_process1(water, water_st)

/var/folders/s8/kn1s0g_s407d23sqsmvzcql80000gn/T/ipykernel_39382/2190094628.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  water['station'] = water['station'].str.replace(r'\(電\)', '')
/var/folders/s8/kn1s0g_s407d23sqsmvzcql80000gn/T/ipykernel_39382/2190094628.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  bool_ = water_st['観測所名称'].str.contains(st)


In [187]:
water_st_p1.query("river.str.contains('太田川')")

,id,station,river,フリガナ,水系名,事務所,市町,データ所管,住所,緯度,経度,入力時使用,評価対象
7,7,三篠橋(国),旧太田川,ミササバシ,太田川,西部建設事務所,広島市中区,国土交通省,広島市中区基町,34.407222,132.450556,1.0,1.0
38,38,中野,太田川,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,39,中野(国),太田川,ナカノ,太田川,西部建設事務所,広島市安佐北区,国土交通省,広島市安佐北区可部,34.508889,132.500833,1.0,1.0
53,53,加計(国),太田川,カケ,太田川,安芸太田支所,安芸太田町,国土交通省,山県郡安芸太田町中ノ渡,34.608889,132.316944,1.0,1.0
70,70,土居(国),太田川,ドイ,太田川,安芸太田支所,安芸太田町,国土交通省,山県郡安芸太田町土居,34.573611,132.218056,1.0,1.0
134,134,江波(旧太田川),旧太田川,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135,135,江波(旧太田川)(国),旧太田川,エバ（キュウオオタガワ）,太田川,西部建設事務所,広島市中区,国土交通省,広島市中区江波南,34.361667,132.418056,1.0,1.0
147,147,玖村(国),太田川,クムラ,太田川,西部建設事務所,広島市安佐北区,国土交通省,広島市安佐北区落合,34.484167,132.500833,1.0,1.0
155,155,矢口第一(国),太田川,ヤグチダイイチ,太田川,西部建設事務所,広島市安佐北区,国土交通省,広島市安佐北区口田,34.461667,132.484167,1.0,1.0
156,156,矢口第二(国),太田川,ヤグチダイニ,太田川,西部建設事務所,広島市安佐北区,国土交通省,広島市安佐北区口田,34.471111,132.484444,1.0,1.0


In [189]:
water_st.query("河川名.str.contains('太田川')")

,観測所名称,フリガナ,水系名,河川名,事務所,市町,データ所管,住所,緯度,経度,入力時使用,評価対象
122,三篠橋(国),ミササバシ,太田川,旧太田川,西部建設事務所,広島市中区,国土交通省,広島市中区基町,34.407222,132.450556,1,1
123,江波(旧太田川)(国),エバ（キュウオオタガワ）,太田川,旧太田川,西部建設事務所,広島市中区,国土交通省,広島市中区江波南,34.361667,132.418056,1,1
131,土居(国),ドイ,太田川,太田川,安芸太田支所,安芸太田町,国土交通省,山県郡安芸太田町土居,34.573611,132.218056,1,1
132,加計(国),カケ,太田川,太田川,安芸太田支所,安芸太田町,国土交通省,山県郡安芸太田町中ノ渡,34.608889,132.316944,1,1
133,飯室(国),イムロ,太田川,太田川,西部建設事務所,広島市安佐北区,国土交通省,広島市安佐北区安佐町毛木,34.535278,132.433333,1,1
134,中野(国),ナカノ,太田川,太田川,西部建設事務所,広島市安佐北区,国土交通省,広島市安佐北区可部,34.508889,132.500833,1,1
135,玖村(国),クムラ,太田川,太田川,西部建設事務所,広島市安佐北区,国土交通省,広島市安佐北区落合,34.484167,132.500833,1,1
136,矢口第二(国),ヤグチダイニ,太田川,太田川,西部建設事務所,広島市安佐北区,国土交通省,広島市安佐北区口田,34.471111,132.484444,1,1
137,矢口第一(国),ヤグチダイイチ,太田川,太田川,西部建設事務所,広島市安佐北区,国土交通省,広島市安佐北区口田,34.461667,132.484167,1,1
138,長和久(国),ナガワク,太田川,太田川,西部建設事務所,広島市安佐南区,国土交通省,広島市安佐南区長束,34.426389,132.451111,1,1


In [181]:
set(water_st['河川名'].unique()) - set(water['river'].unique())

{'太田川\n放水路'}

In [182]:
water_db01= pd.read_csv(Path.cwd().parents[2] / 'data/database01/water_data.csv')
water_st_db01 = pd.read_csv(Path.cwd().parents[2] / 'data/database01/water_station.csv')
print(water_p1.equals(water_db01))
print(water_st_p1.equals(water_st_db01)) # riverの処理はriverでやるのでfalseでいい

True
False


In [152]:
def tide_process1(tide, tide_st):
    tide.loc[tide['station'] == '柿浦漁港', 'station'] = '柿浦港'
    tide.loc[tide['station'] == '呉阿賀港', 'station'] = '呉(阿賀)港'
    tide.loc[tide['station'] == '倉橋漁港', 'station'] = '倉橋港'

    keys = tide.groupby(['station', 'city']).count().index
    tide_db = pd.DataFrame(index=keys).reset_index()
    tide_db['id'] = range(len(tide_db))
    tide_db = tide_db.reindex(columns=['id', 'station', 'city'])

    tide = tide_db.merge(tide, on=['station', 'city'], how='left')
    tide.drop(['station', 'city'], axis=1, inplace=True)

    tide_st = tide_st.rename(columns={'観測所名': 'station'})
    tide_st = tide_db.merge(tide_st, on=['station'], how='left')
    
    return tide, tide_st

In [153]:
tide_p1, tide_st_p1 = tide_process1(tide, tide_st)

In [154]:
tide_db01= pd.read_csv(Path.cwd().parents[2] / 'data/database01/tide_data.csv')
tide_st_db01 = pd.read_csv(Path.cwd().parents[2] / 'data/database01/tide_station.csv')
print(tide_p1.equals(tide_db01))
print(tide_st_p1.equals(tide_st_db01)) # riverの処理はriverでやるのでfalseでいい

True
True


In [155]:
def convert_timeseries(df):
    all_id = df['id'].unique()
    all_date = df['date'].unique().astype(int)
    all_id.sort()
    all_date.sort()
    null_value = 'x'

    data = []
    for st in all_id:
        for date in all_date:
            data.append([date, st])
    
    old_df = df.copy()
    old_df = old_df[~old_df[['date', 'id']].duplicated()]
    old_df.fillna(null_value, inplace=True)

    new_df = pd.DataFrame(data, columns=['date', 'id'])
    new_df = pd.merge(new_df, old_df, on=['date', 'id'], how='left')
    new_df = new_df.sort_values(['id', 'date']).reset_index(drop=True)
    del old_df
    
    series = new_df.iloc[:, 2:].values.reshape(472, -1).T
    timeseries_df = pd.DataFrame(series, columns=all_id)

    dates = []
    hours = []
    for date in all_date:
        for hour in range(24):
            dates.append(date)
            hours.append(hour)
    timeseries_df['date'] = dates
    timeseries_df['hour'] = hours
    timeseries_df = timeseries_df.reindex(columns=['date', 'hour'] + list(all_id))
    return timeseries_df

In [169]:
water.query('river.str.contains("太田川放水路")')

,date,station,river,00:00:00,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,...,14:00:00,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00
14,0,草津(国),太田川放水路,-0.26,-0.91,-1.5,-1.77,-1.5,-0.84,-0.08,...,-0.22,-0.65,-0.81,-0.44,0.17,0.74,1.17,1.48,1.43,0.86
193,1,草津(国),太田川放水路,0.09,-0.59,-1.29,-1.87,-2.04,-1.66,-0.88,...,0.04,-0.47,-0.87,-0.92,-0.46,0.21,0.81,1.25,1.51,1.34
372,2,草津(国),太田川放水路,0.68,-0.04,-0.77,-1.51,-1.98,-2.06,-1.55,...,0.55,-0.07,-0.56,-0.94,-0.86,-0.27,0.42,1,1.38,1.61
551,3,草津(国),太田川放水路,1.33,0.62,-0.1,-0.81,-1.47,-1.91,-1.86,...,1.01,0.28,-0.27,-0.73,-1.02,-0.79,-0.14,0.48,0.99,1.34
730,4,草津(国),太田川放水路,1.49,1.09,0.36,-0.26,-0.92,-1.52,-1.86,...,1.29,0.69,0.06,-0.45,-0.91,-1.06,-0.65,-0.04,0.53,0.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391220,2186,草津(国),太田川放水路,0.53,-0.11,-0.8,-1.36,-1.67,-1.42,-0.74,...,0.41,-0.01,-0.38,-0.49,-0.2,0.32,0.81,1.17,1.38,1.26
391399,2187,草津(国),太田川放水路,0.71,0.09,-0.52,-1.13,-1.59,-1.66,-1.2,...,0.69,0.18,-0.22,-0.54,-0.5,-0.03,0.49,0.94,1.21,1.31
391578,2188,草津(国),太田川放水路,0.98,0.38,-0.2,-0.77,-1.35,-1.65,-1.47,...,0.99,0.45,0.05,-0.32,-0.55,-0.31,0.22,0.65,1.01,1.18
391757,2189,草津(国),太田川放水路,1.16,0.77,0.19,-0.31,-0.84,-1.32,-1.42,...,1.25,0.76,0.34,0,-0.38,-0.4,-0.01,0.42,0.81,1.05


In [156]:
water_st_p1.query('river == "太田川放水路"')

,id,station,river,フリガナ,水系名,事務所,市町,データ所管,住所,緯度,経度,入力時使用,評価対象
174,174,草津(国),太田川放水路,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [163]:
water_st.query('河川名.str.contains("太田川")')

,観測所名称,フリガナ,水系名,河川名,事務所,市町,データ所管,住所,緯度,経度,入力時使用,評価対象
122,三篠橋(国),ミササバシ,太田川,旧太田川,西部建設事務所,広島市中区,国土交通省,広島市中区基町,34.407222,132.450556,1,1
123,江波(旧太田川)(国),エバ（キュウオオタガワ）,太田川,旧太田川,西部建設事務所,広島市中区,国土交通省,広島市中区江波南,34.361667,132.418056,1,1
131,土居(国),ドイ,太田川,太田川,安芸太田支所,安芸太田町,国土交通省,山県郡安芸太田町土居,34.573611,132.218056,1,1
132,加計(国),カケ,太田川,太田川,安芸太田支所,安芸太田町,国土交通省,山県郡安芸太田町中ノ渡,34.608889,132.316944,1,1
133,飯室(国),イムロ,太田川,太田川,西部建設事務所,広島市安佐北区,国土交通省,広島市安佐北区安佐町毛木,34.535278,132.433333,1,1
134,中野(国),ナカノ,太田川,太田川,西部建設事務所,広島市安佐北区,国土交通省,広島市安佐北区可部,34.508889,132.500833,1,1
135,玖村(国),クムラ,太田川,太田川,西部建設事務所,広島市安佐北区,国土交通省,広島市安佐北区落合,34.484167,132.500833,1,1
136,矢口第二(国),ヤグチダイニ,太田川,太田川,西部建設事務所,広島市安佐北区,国土交通省,広島市安佐北区口田,34.471111,132.484444,1,1
137,矢口第一(国),ヤグチダイイチ,太田川,太田川,西部建設事務所,広島市安佐北区,国土交通省,広島市安佐北区口田,34.461667,132.484167,1,1
138,長和久(国),ナガワク,太田川,太田川,西部建設事務所,広島市安佐南区,国土交通省,広島市安佐南区長束,34.426389,132.451111,1,1


In [161]:
water_st_p1.query('river.str.contains("太田川")')

,id,station,river,フリガナ,水系名,事務所,市町,データ所管,住所,緯度,経度,入力時使用,評価対象
7,7,三篠橋(国),旧太田川,ミササバシ,太田川,西部建設事務所,広島市中区,国土交通省,広島市中区基町,34.407222,132.450556,1.0,1.0
38,38,中野,太田川,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,39,中野(国),太田川,ナカノ,太田川,西部建設事務所,広島市安佐北区,国土交通省,広島市安佐北区可部,34.508889,132.500833,1.0,1.0
53,53,加計(国),太田川,カケ,太田川,安芸太田支所,安芸太田町,国土交通省,山県郡安芸太田町中ノ渡,34.608889,132.316944,1.0,1.0
70,70,土居(国),太田川,ドイ,太田川,安芸太田支所,安芸太田町,国土交通省,山県郡安芸太田町土居,34.573611,132.218056,1.0,1.0
134,134,江波(旧太田川),旧太田川,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135,135,江波(旧太田川)(国),旧太田川,エバ（キュウオオタガワ）,太田川,西部建設事務所,広島市中区,国土交通省,広島市中区江波南,34.361667,132.418056,1.0,1.0
147,147,玖村(国),太田川,クムラ,太田川,西部建設事務所,広島市安佐北区,国土交通省,広島市安佐北区落合,34.484167,132.500833,1.0,1.0
155,155,矢口第一(国),太田川,ヤグチダイイチ,太田川,西部建設事務所,広島市安佐北区,国土交通省,広島市安佐北区口田,34.461667,132.484167,1.0,1.0
156,156,矢口第二(国),太田川,ヤグチダイニ,太田川,西部建設事務所,広島市安佐北区,国土交通省,広島市安佐北区口田,34.471111,132.484444,1.0,1.0
